In [74]:
from pathlib import Path
import ffmpeg
import argparse
import numpy as np
import logging
import pandas as pd
import math
import numpy as np

class Sampler:
    def __init__(self, input_path, ann_file_path):
        """
        The Sampler class contains sampling methods for the video input data.
        :param input_path: file path for the input videos
        :param ann_file_path: file path for annotation file belonging to the video input data
        """
        self.input = input_path
        self.ann_file = ann_file_path

    
    def stratifiedSampling(self, output_path, trim_len=10, n_fall_samples=3, samples_per_min=1):
        """
        Stratified sampling only samples from videos which contain all three 
        actions (ADL, falling, lying). From these videos, a base rate of fall 
        samples will be sampled uniformly on the fall interval (time where fall happens). 
        ADL and lying activities are sampled based on their ratio of duration based 
        on sample rate.
        :param trim_len: sample video length in seconds (defaults to 10) 
        :param fall_samples: amount of fall samples to collect from videos where 
        falls occur (defaults to 3) 
        :param samples_per_min: amount of samples per minute from ADL and lying activities 
        (defaults to 1)
        :param output_path: filepath for sample outputs
        """
        
        # Read annotation file to dataframe
        df = pd.read_csv(self.ann_file)
        
        # Filter for only fall videos
        df = df[df['category'] == "Fall"] 
        
        # Reset index after filtering
        df = df.reset_index()
        
        sample_list = []

        for i in df.index:
            
            # Get timestamps
            fall_start = float(df.loc[i, 'fall_start'])
            fall_end = float(df.loc[i, 'fall_end'])
            lying_start = float(df.loc[i, 'lying_start'])
            lying_end = float(df.loc[i, 'lying_end'])
            video_end = float(df.loc[i, 'length'])
            
            # Calculate action durations
            ADL1_time = fall_start
            ADL2_time = video_end - lying_end
            ADL_time = ADL1_time + ADL2_time
            fall_time = fall_end - fall_start
            lying_time = lying_end - lying_start
            
            # Calculate number of samples for ADL and lying activities
            n_samples = round((ADL_time + lying_time) / 60) * samples_per_min
            n_ADL_samples = round(n_samples * (ADL_time / (ADL_time + lying_time)))
            n_lying_samples = n_samples - n_ADL_samples
            n_ADL1_samples = round((ADL1_time/ADL_time)*n_ADL_samples)
            n_ADL2_samples = n_ADL_samples - n_ADL1_samples
            
            # Sample uniformly on the intervals
            ADL1_samples = np.random.uniform(0, ADL1_time, n_ADL1_samples)
            ADL2_samples = np.random.uniform(lying_end, video_end, n_ADL2_samples)
            fall_samples = np.random.uniform(fall_start, fall_end, n_fall_samples)
            lying_samples = np.random.uniform(lying_start, lying_end, n_lying_samples)
            
            # Create sample list [video path, [sample timestamps]]
            sample_list.append([
                df.loc[i, "video_path"], 
                np.concatenate((ADL1_samples, 
                               ADL2_samples, 
                               fall_samples, 
                               lying_samples)).tolist()
            ])
            
        # Generate samples
        self.outputSamples(sample_list, output_path)
    
    
    def outputSamples(self, sample_list, output_path, trim_len=10):
        """
        Utility function for trimming input videos and outputting them
        to the given output path (generating samples).
        :param sample_list: a list containing the video name and a list 
        of sample start timestamps, e.g. 
        [data/Fall_Simulation_Data/videos/Fall30_Cam3.avi, 
         [7.16443459, 15.836356, 104.36721318, 26.32500079]
         ]
        :param output_path: filepath for sample outputs
        :param trim_len: sample video length in seconds (defaults to 10) 
        """
        logging.basicConfig(level=logging.INFO)
        logger = logging.getLogger(__name__)
        
        #TODO: remove
        count = 0  
        
        for sample in sample_list:
            
            # TODO: remove
            if count > 0:
                break
            count += 1
            
             # Create path
            path = Path("../" + sample[0])
            output_path = Path(output_path)
            output_path.mkdir(parents=True, exist_ok=True)
            
            # Store sample timestamps
            timestamps = sample[1]
            print(f'Timestamps: {timestamps}')
            
            # Get video data
            video_probe = ffmpeg.probe(path)
            video_duration = video_probe.get("format", {}).get("duration", None)
            logger.debug(f"Video duration: {video_duration}")
            input_stream = ffmpeg.input(path)
            
            # Output samples
            for t in timestamps:
                
                # Trim video
                video = input_stream.trim(start=t, end=t+trim_len).setpts(
                    "PTS-STARTPTS"
                )
                
                # Create output path
                output_file_path = output_path.joinpath(
                    Path(sample[0]).stem + f"_{round(t, 3)}_{round(t+trim_len, 3)}" + ".mp4"
                )  # e.g. output_path/ADL1_Cam2_20_30.avi

                # Output
                output = ffmpeg.output(video, output_file_path.as_posix(), f="mp4")
                output.run()
                
        logger.info("Videos trimming completed successfully.")

In [75]:
strat = Sampler("nothingfornow", "../data/Fall_Simulation_Data/annotations.csv")

strat.stratifiedSampling("../data/Fall_Simulation_Data/sample_outputs/", samples_per_min=3)

Timestamps: [21.86586029763899, 13.64965263837777, 93.39968160459098, 25.42596113021942, 26.59465366043841, 24.46918285662431, 43.4301568313621, 60.29304291621934, 61.24955984088418]


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57

frame=  300 fps=0.0 q=-1.0 Lsize=     397kB time=00:00:09.90 bitrate= 328.1kbits/s speed=11.3x    
video:392kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.103588%
[libx264 @ 0x131815000] frame I:2     Avg QP:16.08  size: 30048
[libx264 @ 0x131815000] frame P:76    Avg QP:19.92  size:  3995
[libx264 @ 0x131815000] frame B:222   Avg QP:18.07  size:   168
[libx264 @ 0x131815000] consecutive B-frames:  1.0%  0.0%  3.0% 96.0%
[libx264 @ 0x131815000] mb I  I16..4: 37.2% 55.0%  7.8%
[libx264 @ 0x131815000] mb P  I16..4:  0.4%  0.7%  0.0%  P16..4: 27.8%  5.0%  5.2%  0.0%  0.0%    skip:60.9%
[libx264 @ 0x131815000] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 11.8%  0.0%  0.0%  direct: 0.0%  skip:88.1%  L0:46.8% L1:52.7% BI: 0.5%
[libx264 @ 0x131815000] 8x8 transform intra:57.4% inter:93.3%
[libx264 @ 0x131815000] coded y,uvDC,uvAC intra: 51.5% 60.7% 37.5% inter: 2.8% 4.4% 1.3%
[libx264 @ 0x131815000] i16 v,h,dc,p: 45% 15%  3% 37%
[libx264 @ 0x131815000] i8 v,h,dc

frame=  300 fps=0.0 q=-1.0 Lsize=     408kB time=00:00:09.90 bitrate= 337.4kbits/s speed=20.6x    
video:403kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.076609%
[libx264 @ 0x144815000] frame I:2     Avg QP:16.07  size: 31897
[libx264 @ 0x144815000] frame P:76    Avg QP:20.11  size:  4067
[libx264 @ 0x144815000] frame B:222   Avg QP:18.26  size:   178
[libx264 @ 0x144815000] consecutive B-frames:  1.0%  0.7%  1.0% 97.3%
[libx264 @ 0x144815000] mb I  I16..4: 35.9% 53.3% 10.8%
[libx264 @ 0x144815000] mb P  I16..4:  0.4%  0.4%  0.0%  P16..4: 28.3%  4.9%  5.3%  0.0%  0.0%    skip:60.7%
[libx264 @ 0x144815000] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 12.5%  0.0%  0.0%  direct: 0.0%  skip:87.5%  L0:52.2% L1:47.2% BI: 0.5%
[libx264 @ 0x144815000] 8x8 transform intra:53.2% inter:93.3%
[libx264 @ 0x144815000] coded y,uvDC,uvAC intra: 49.4% 60.0% 39.2% inter: 2.9% 4.5% 1.3%
[libx264 @ 0x144815000] i16 v,h,dc,p: 45% 14%  3% 38%
[libx264 @ 0x144815000] i8 v,h,dc

frame=  300 fps=0.0 q=-1.0 Lsize=     388kB time=00:00:09.90 bitrate= 321.4kbits/s speed=  18x    
video:384kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.130975%
[libx264 @ 0x128815000] frame I:2     Avg QP:15.96  size: 31726
[libx264 @ 0x128815000] frame P:76    Avg QP:19.82  size:  3906
[libx264 @ 0x128815000] frame B:222   Avg QP:17.15  size:   146
[libx264 @ 0x128815000] consecutive B-frames:  1.3%  0.0%  0.0% 98.7%
[libx264 @ 0x128815000] mb I  I16..4: 36.5% 55.0%  8.5%
[libx264 @ 0x128815000] mb P  I16..4:  0.4%  0.6%  0.0%  P16..4: 27.5%  4.5%  5.0%  0.0%  0.0%    skip:62.0%
[libx264 @ 0x128815000] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 10.4%  0.0%  0.0%  direct: 0.0%  skip:89.5%  L0:39.7% L1:59.8% BI: 0.6%
[libx264 @ 0x128815000] 8x8 transform intra:56.9% inter:94.0%
[libx264 @ 0x128815000] coded y,uvDC,uvAC intra: 52.1% 61.9% 39.1% inter: 2.7% 4.1% 1.3%
[libx264 @ 0x128815000] i16 v,h,dc,p: 45% 15%  3% 37%
[libx264 @ 0x128815000] i8 v,h,dc

frame=  300 fps=0.0 q=-1.0 Lsize=     456kB time=00:00:09.90 bitrate= 377.7kbits/s speed=14.8x    
video:452kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.960843%
[libx264 @ 0x12980ca00] frame I:2     Avg QP:16.45  size: 30715
[libx264 @ 0x12980ca00] frame P:76    Avg QP:20.55  size:  4708
[libx264 @ 0x12980ca00] frame B:222   Avg QP:18.92  size:   194
[libx264 @ 0x12980ca00] consecutive B-frames:  1.3%  0.0%  0.0% 98.7%
[libx264 @ 0x12980ca00] mb I  I16..4: 37.9% 52.4%  9.7%
[libx264 @ 0x12980ca00] mb P  I16..4:  0.4%  0.3%  0.0%  P16..4: 31.6%  5.9%  6.6%  0.0%  0.0%    skip:55.1%
[libx264 @ 0x12980ca00] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 14.1%  0.0%  0.0%  direct: 0.0%  skip:85.8%  L0:45.3% L1:54.3% BI: 0.4%
[libx264 @ 0x12980ca00] 8x8 transform intra:51.7% inter:92.9%
[libx264 @ 0x12980ca00] coded y,uvDC,uvAC intra: 46.8% 57.1% 35.7% inter: 3.4% 5.3% 1.5%
[libx264 @ 0x12980ca00] i16 v,h,dc,p: 45% 16%  2% 36%
[libx264 @ 0x12980ca00] i8 v,h,dc

### 

# Saved print-outs
print('ADL1 time: ', ADL1_time)
print('ADL2 time: ', ADL2_time)
print('Total ADL time: ', ADL_time)
print('Fall time: ', fall_time)
print('Lying time: ', lying_time)

print("n samples: ", n_samples)
print("n ADL samples: ", n_ADL_samples)
print("n lying samples: ", n_lying_samples)
print("n ADL1 samples: ", n_ADL1_samples)
print("n ADL2 samples: ", n_ADL2_samples)

print("ADL1_samples: ", ADL1_samples)
print("ADL2_samples: ", ADL2_samples)
print("fall_samples: ", fall_samples)
print("lying_samples: ", lying_samples)

print(f'Video path: {df.loc[i, "video_path"]}')
print("ADL1_samples: ", ADL1_samples)
print("ADL2_samples: ", ADL2_samples)
print("fall_samples: ", fall_samples)
print("lying_samples: ", lying_samples)
print()